In [ ]:
import altair as alt
import polars as pl

alt.renderers.enable("browser")

In [ ]:
df_units = pl.read_csv("../data/units.csv")


In [ ]:
arrows_dict = []
for row in df_units.iter_rows(named=True):
    if not row["is_base_unit"]:
        mults = row["mult_depencendy"].split(",")
        for _m in mults:
            if _m != "":
                _row = df_units.filter(pl.col("unit_name") == _m)["row"][0]
                _col = df_units.filter(pl.col("unit_name") == _m)["column"][0]
                _name = df_units.filter(pl.col("unit_name") == _m)["unit_name"][
                    0
                ]

                arrows_dict.append(
                    {
                        "unit_name": _name,  # row["unit_name"],
                        "type": "MULTIPLICATION",
                        "color": _name,
                        "start": _name,
                        "start_column": _col,
                        "start_row": _row,
                        "end": row["unit_name"],
                        "end_column": row["column"],
                        "end_row": row["row"],
                        "start_end": _name + " " + row["unit_name"],
                    }
                )
        divs = row["divide_dependency"].split(",")
        for _d in divs:
            if _d != "":
                _row = df_units.filter(pl.col("unit_name") == _d)["row"][0]
                _col = df_units.filter(pl.col("unit_name") == _d)["column"][0]
                _name = df_units.filter(pl.col("unit_name") == _d)["unit_name"][
                    0
                ]

                arrows_dict.append(
                    {
                        "unit_name": _name,  # row["unit_name"],
                        "type": "DIVISION",
                        "color": _name,
                        "start": _name,
                        "start_column": _col,
                        "start_row": _row,
                        "end": row["unit_name"],
                        "end_column": row["column"],
                        "end_row": row["row"],
                        "start_end": _name + " " + row["unit_name"],
                    }
                )

df_arrows = (
    (
        pl.DataFrame(arrows_dict)
        # .with_columns(
        #     dx=pl.col("end_column") - pl.col("start_column"),
        #     dy=pl.col("end_row") - pl.col("start_row"),
        # )
        # .with_columns(len=(pl.col("dx") ** 2 + pl.col("dy") ** 2).sqrt())
        # .with_columns(
        #     start_column2=pl.col("start_column")
        #     + 0.3 * pl.col("dx") / pl.col("len"),
        #     start_row2=pl.col("start_row") + 0.3 * pl.col("dy") / pl.col("len"),
        #     end_column2=pl.col("end_column")
        #     - 0.3 * pl.col("dx") / pl.col("len"),
        #     end_row2=pl.col("end_row") - 0.3 * pl.col("dy") / pl.col("len"),
        # )
    ).select(
        "unit_name",
        "type",
        "start_column",
        "start_row",
        "end_column",
        "end_row",
        "start_end",
    )
    # .filter(pl.col("unit_name") == "lumen")
)

df_arrows

In [ ]:
df = df_units.join(df_arrows, on="unit_name", how="left", coalesce=True)


In [ ]:
base = alt.Chart(df)  # .encode(tooltip=["unit_name", "description"])
chart_base_base_units = base.encode(
    x=alt.X("column:Q").axis(None).scale(domain=[0, 6]),
    y=alt.Y("row:Q").axis(None).scale(domain=[0, 6], reverse=True),
).transform_filter(alt.datum.is_base_unit)
chart_base_derived_units = base.encode(
    x=alt.X("column:Q").axis(None).scale(domain=[0, 6]),
    y=alt.Y("row:Q").axis(None).scale(domain=[0, 6], reverse=True),
).transform_filter(~alt.datum.is_base_unit)
colors = (
    alt.Color("unit_name:N")
    .scale(
        range=df_units["color"].to_list(),
        domain=df_units["unit_name"].to_list(),
    )
    .legend(None)
)

chart = (
    alt.layer(
        #
        # ARROWS
        #
        base.mark_line(strokeWidth=4).encode(
            # detail="start_end:N",
            x=alt.X("start_column:Q").axis(None).scale(domain=[0, 6]),
            y=alt.Y("start_row:Q")
            .axis(None)
            .scale(domain=[0, 6], reverse=True),
            x2=alt.X2("end_column:Q"),
            y2=alt.Y2("end_row:Q"),
            detail="start_end:N",
            color=colors,
            strokeDash=alt.Stroke("type:N")
            .scale(  # type: ignore
                domain=["MULTIPLICATION", "DIVISION"], range=[[], [5, 5]]
            )
            .legend(None),
        ),
        # chart_base_arrow.mark_text().encode(text="start_end:N"),
        #
        # BASE UNITS
        #
        chart_base_base_units.mark_point(
            filled=True, size=20000, shape="square", opacity=1
        ).encode(color=colors),
        chart_base_base_units.mark_text(
            fontSize=50, dy=0, fontWeight="bold"
        ).encode(
            text="unit_symbol:N",
            color=alt.value("white"),
        ),
        chart_base_base_units.mark_text(
            fontSize=20, lineBreak=r"\n", dy=-60, fontWeight="bold"
        ).encode(text="unit_name:N", color=colors),
        #
        # NON BASE UNITS
        #
        chart_base_derived_units.mark_point(
            filled=True, size=25000, strokeWidth=3, opacity=1
        ).encode(color=alt.value("white")),
        chart_base_derived_units.mark_point(
            filled=False, size=25000, strokeWidth=5, opacity=1
        ).encode(color=alt.value("gray")),
        chart_base_derived_units.mark_text(fontSize=25, dy=-45).encode(
            text="unit_symbol:N",
        ),
        chart_base_derived_units.mark_text(
            fontSize=13, lineBreak=r"\n", dy=-25
        ).encode(text="unit_name:N", color=alt.value("grey")),
        chart_base_derived_units.mark_text(
            fontSize=13, lineBreak=r"\n", dy=5
        ).encode(text="base_units_composite:N", color=alt.value("grey")),
        chart_base_derived_units.mark_text(
            fontSize=10, lineBreak=r"\n", dy=30
        ).encode(text="description:N", color=alt.value("grey")),
    )
    .properties(width=340 * 6, height=340 * 4)
    .configure_axis(grid=False)
    .configure_view(stroke=None)  # type: ignore
)
chart.show()

In [ ]:
chart.save("chart.html")